# Spark assignment 1: Pairs
Find all the pairs of two consequent words where the first word is “narodnaya”. For each pair, count the number of occurrences in the Wikipedia dump. Print all the pairs with their count in a lexicographical order. Output format is “word_pair <tab> count”, for example:

<code>
    red_apple	100500
    crazy_zoo	42
</code>

Note that two words in a pair are concatenated with the underscore character, and the result is in the lowercase.

One motivation for counting these continuations is to get a better understanding of the language. Some words, like “the”, have a lot of continuations, while others, like “San”, have just a few (“San Francisco”, for example). One can build a language model with these statistics. If you are interested to learn more, search for “n-gram language model” in the Internet.

Dataset location: /data/wiki/en_articles_part

The result on the sample dataset:

<code>
    narodnaya_gazeta 1
    narodnaya_volya 9
</code>

In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [ ]:
# ! cat "/data/wiki/en_articles_part/articles-part" | head -1

In [35]:
import re

def split_words(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []
    
def make_pairs(words):
    last_word = None
    for word in words:
        word = word.lower()
        if last_word and last_word == "narodnaya":
            yield "%s_%s" % (last_word, word)
        last_word = word

query = sc.textFile("/data/wiki/en_articles_part/articles-part", 16) \
    .map(split_words) \
    .flatMap(make_pairs) \
    .map(lambda x: (str(x), 1)) \
    .reduceByKey(lambda x, y: x + y)

result = query.collect()
for key, count in result:
    print("%s\t%d" % (key, count))

narodnaya_gazeta	1
narodnaya_volya	9
